In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [2]:
N = 6
a_x = 0
b_x = 1
a_y = 0
b_y = 1
h = (b_x-a_x)/(N - 1) # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

In [3]:
def init_U():
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U

U = init_U()

U_expected = U.copy() # задаём границы
U_expected[1:iN+1,1:iN+1] = np.array([[u_expected(i*h,j*h) for j in range(1, iN+1)] for i in range(1, iN+1)])

F = np.array([[f(i*h, j*h) for j in range(1, iN+1)] for i in range(1,iN+1)])

In [13]:
class DESolver:
    def __init__(self, grid_step, N, eps = 1e-5):
        self.h = grid_step
        self.methods = {'iteration': self.iteration_method_iter,
                        'seidel': self.seidel_method_iter,
                        'over-relaxation': self.overrelaxation_method_iter}
        self.methods_params = {'iteration'      : {'min_iters': 2*np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)},
                               'seidel'         : {'min_iters': np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)**2},
                               'over-relaxation': {'min_iters': 2*np.log(1/eps)/(np.pi*h),
                                                'spectral_rad': np.cos(np.pi * h),
                                                'opt_parameter': 2/(1 + np.sin(np.pi * h))}}
        self.iN = N - 2
        C = np.matrix([[4, -1] + (N-2)*[0]] +
              [ k*[0] + [-1, 4, -1] + (N - 3 - k)*[0] for k in range(0, N - 2)] +
              [(N-2)*[0] + [-1, 4]], dtype=float)
        self.mmatrix = np.block([[C, -np.eye(N = N)] + (N-2)*[np.zeros( shape = (N, N) )]] +
              [ k*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C, -np.eye(N = N)] + (N - 3 - k)*[np.zeros( shape = (N, N) )] for k in range(0, N - 2)] +
              [(N-2)*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C]]) / grid_step**2
        self.lastsol = np.zeros(shape = (N, N))
        self.iter_diffs = np.zeros(shape = 3) # Хранит разницу приближений последних трёх итераций

    def __method_iter(self, U_curr, method_name, method_params):
        return self.methods[method_name](U_curr, params = method_params)

    def iteration_method_iter(self, U, params = None):
        for i in range(1, self.iN+1):
            for j in range(1, self.iN+1):
               self.lastsol[i, j] = 0.25*(U[i-1, j]+U[i+1,j]+U[i, j-1]+U[i, j+1]+F[i-1,j-1]*self.h**2)
        return self.lastsol

    def seidel_method_iter(self, U, params = None):
        for k in range(1, iN+1):
            for i in range(k, iN+1):
                self.lastsol[i, k] = 0.25*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
            for j in range(k+1, iN+1):
                self.lastsol[k, j] = 0.25*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
        return self.lastsol

    def overrelaxation_method_iter(self, U, params):
            for k in range(1, iN+1):
                for i in range(k, iN+1):
                    self.lastsol[i, k] = (1-params['opt'])*U[i,k] + 0.25*params['opt']*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
                for j in range(k+1, iN+1):
                    self.lastsol[k, j] = (1-params['opt'])*U[k,j] + 0.25*params['opt']*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
            return self.lastsol

    def __calculate_props(self, U, U_expected, iter, init_error, init_discrepancy, apost_est_coef):
        iter_diff = np.max(np.abs(U - self.lastsol))
        self.iter_diffs[iter % 3] = iter_diff
        apost_est = apost_est_coef * iter_diff
        spec_rad_est = ' ' if iter < 2 else np.sqrt(self.iter_diffs[iter % 3]/ self.iter_diffs[(iter - 2) % 3] )

        U_reshaped = np.reshape(self.lastsol, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix @ U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

        error = np.max(np.abs(self.lastsol - U_expected))
        discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        rel_discrepancy = discrepancy / init_discrepancy
        rel_error = error / init_error

        return [iter + 1, discrepancy, rel_discrepancy,
                             error, rel_error, iter_diff, apost_est, spec_rad_est]

    def solve(self, U_init, U_expected, eps = 1e-5, method = 'iteration', method_parameters = None):
        if method not in ['iteration', 'seidel', 'over-relaxation']:
            return None
        else:
            self.iter_diffs = np.zeros(shape = 3)
            if method_parameters is None and method == 'over-relaxation':
                method_parameters = {'opt': self.methods_params['over-relaxation']['opt_parameter']}
        #apost_est_coef = map(lambda x: x/1-x, [self.methods_params[method]['spectral_rad']])
        apost_est_coef = self.methods_params[method]['spectral_rad']/ (1 - self.methods_params[method]['spectral_rad'])
        init_error = np.max(np.abs(U_init - U_expected))
        U = U_init.copy()
        self.lastsol = U.copy()

        U_reshaped = np.reshape(U, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix@U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

        init_discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        U_new = U.copy()
        output_list = list()
        iter = 0
        rel_discrepancy = 1

        while rel_discrepancy > eps and iter < self.methods_params[method]['min_iters']:
            U_new = self.__method_iter(U, method, method_parameters)

            curr_iter_props = self.__calculate_props(U, U_expected, iter, init_error, init_discrepancy, apost_est_coef)
            output_list+=[ curr_iter_props ]

            U = U_new.copy()
            rel_discrepancy = curr_iter_props[2]
            iter+=1
                                                                                      
        output_list += [[' ', 'min_iters:', self.methods_params[method]['min_iters'], ' ', ' method:', method, ' ', ' ']]
        return {'solution': U ,'output': pd.DataFrame(data = output_list,
                                                      columns = ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $', 'Apost. est.', '$\hat{\rho}_k$']).set_index(keys = 'k')}


In [14]:
solver = DESolver(h, N)

iter_summary = solver.solve(U, method = 'over-relaxation', U_expected = U_expected)
solU = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,12.818102,0.309844,0.178156,0.339806,0.548197,2.322201,
2,8.068365,0.195031,0.121005,0.230799,0.172424,0.730399,
3,3.882178,0.093841,0.065343,0.124632,0.107656,0.456037,0.44315
4,2.467779,0.059652,0.043363,0.082709,0.051782,0.219351,0.548011
5,1.318445,0.031870,0.020158,0.038447,0.033384,0.141419,0.55687
6,0.8154,0.019710,0.009445,0.018014,0.017576,0.074453,0.582602
7,0.122073,0.002951,0.000877,0.001673,0.010271,0.043508,0.554667
8,0.046935,0.001135,0.000712,0.001357,0.001538,0.006514,0.29578
9,0.006236,0.000151,0.00012,0.000229,0.000591,0.002504,0.239917


In [ ]:
solU[0:N + 1,0:N+1]

In [ ]:
F


In [ ]:
solU
